# Career Compass: AI-Powered Job Pathway Designer with GenAI

This notebook demonstrates a Generative AI application that converts resume or LinkedIn profile data into a personalized roadmap for career transition. It identifies potential new roles, skill gaps, and curated resources.

**Use Case**: Personalized career guidance for students and professionals transitioning between industries.

**GenAI Capabilities Used**:
- Document Understanding
- Few-shot Prompting
- Structured Output (JSON format)
- Grounding (Google Search)

📂 **Before You Run This Notebook:**
By default, to use this notebook you must upload your resume as a PDF file.

1. Click **Add Data** > **Upload** on the right-hand sidebar.
2. Select your resume file and ensure it is named `my_resume.pdf`.
3. The uploaded file will be stored in `/kaggle/input/`.

🔄 **Alternatively**: If you prefer not to upload a resume, you can comment out the PDF cell (Cell 5) and uncomment the LinkedIn cell (Cell 6), then paste in the public URL to your LinkedIn profile. Note that many LinkedIn pages require authentication and may not always load successfully in this environment.

# Install Dependencies

In [ ]:
!pip install -q -U google-genai

# Import Libraries

In [ ]:
import os
import json
import pathlib
import httpx
from pprint import pprint
from google import genai
from google.genai import types
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# Set API Key

In [ ]:
# Code snippet from Kaggle Secrets Manager
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise EnvironmentError("Gemini API key not found. Please set the GEMINI_API_KEY environment variable.")

client = genai.Client(api_key=GEMINI_API_KEY)

# Specify Target Role and Sector 

In [ ]:
# Enter the job title and sector you're targeting
target_role = "Product Manager"
target_sector = "Technology"

# Define JSON Schema and Prompt Template

In [ ]:
json_schema = '''
CareerPath = {
  "transferable_skills": list[{
    "name": str,
    "evidence": str
  }],
  "missing_skills": list[str],
  "recommended_certifications": list[str],
  "learning_resources": list[str],
  "intermediate_roles": list[str],
  "roadmap_summary": str
}
Return: CareerPath as JSON
'''

base_prompt = f"""
The user wants to transition into a career as a {target_role} in the {target_sector} sector.
Analyze their background and:
- Identify transferable skills they already have, and cite the specific sentence or phrase from their resume or profile that supports each skill
- Identify key skills and certifications they are missing
- Recommend learning resources
- Suggest a sequence of intermediate roles they could pursue to transition toward their target, in the order of role hierarchy (lowest to highest)
- Provide a one-paragraph summary of their roadmap
Format the output using this schema:
{json_schema}
"""

# Upload and Process Local Resume PDF

In [ ]:
# Ensure 'file.pdf' is uploaded to your notebook before running.

from pathlib import Path

filepath = Path("/kaggle/input/.../file.pdf")
if not filepath.exists():
    raise FileNotFoundError(" 'file.pdf' was not found. Please upload your resume using the 'Add Data > Upload' menu before running this notebook.")

pdf_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[
        types.Part.from_bytes(
            data=filepath.read_bytes(),
            mime_type="application/pdf",
        ),
        base_prompt
    ]
)

# Extract and Parse JSON Career Plan from AI Response

In [ ]:
try:
    # Extract the JSON from the response text
    response_text = pdf_response.text
    
    # Look for JSON content within markdown code blocks
    if "```json" in response_text:
        # Extract content between ```json and ```
        json_text = response_text.split("```json")[1].split("```")[0].strip()
    else:
        # If no markdown formatting, use the raw text
        json_text = response_text
    
    # Now parse the extracted JSON content
    career_plan = json.loads(json_text)
    
    # Print the parsed career plan to verify
    print("Successfully parsed career plan:")
    pprint(career_plan)
    
except Exception as e:
    raise ValueError(f"Failed to parse AI response as JSON: {e}")

# Find Learning Resources with Google Search Grounding & Few-Shot Prompting

In [ ]:
# Use Google's built-in search tool
model_id = "gemini-2.0-flash"
google_search_tool = Tool(google_search=GoogleSearch())

examples ="""
[{
  "skill": "Product Strategy (developing product vision and roadmaps)",
  "resources": [
    {
      "title": "Product Strategy: Develop and Manage Successful Products",
      "provider": "Kellogg School of Management",
      "description": "Designed to train business stakeholders. The course content will enable executives to prepare wireframes, decide pricing models for SaaS, and guide product launch and subsequent revisions for growth strategies."
    },
    {
        ...
    },
    {
      "title": "Mastering Product Management: Building Your Strategy",
      "provider": "Stanford Online",
      "description": "Provides a structured playbook for developing product strategies aligned with company goals. It covers essential techniques such as creating roadmaps, managing product teams, and driving product growth."
    }
  ]
}],
[{
  "skill": "Market Research (analyzing market trends and competitive landscape)",
  "resources": [
    {
      "title": "Market Research Specialization",
      "provider": "University of California, Davis on Coursera",
      "description": "Offers a comprehensive approach to market research, suitable for various experience levels."
    },
    {
        ...
    },
    {
      "title": "Market Research and Consumer Behavior",
      "provider": "IE Business School",
      "description": "Focuses on understanding consumer behavior and how it impacts market research."
    }
  ]
}],
...,
[{
  "skill": "User Research (understanding user needs and pain points)",
  "resources": [
    {
      "title": "Google UX Design Certificate",
      "provider": "Coursera",
      "description": "A beginner-friendly course that covers the basics of user experience design, including user research, ideation, wireframing, and prototyping."
    },
    {
        ...
    },
    {
      "title": "User Research – Methods and Best Practices",
      "provider": "The Interaction Design Foundation",
      "description": "Provides in-depth training on user research methods, focusing on practical applications for product and UX design."
    }
  ]
}]
"""

main_query = f"""Return this in JSON format matching my examples exactly. For each of the following skills, list 2-3 reputable online courses, certifications or other learning resources:

Skills: {', '.join(career_plan['missing_skills'])}

The output should be a JSON array where each element follows this structure:
[{{
  "skill": "[Skill Name]",
  "resources": [
    {{
      "title": "[Resource Title]",
      "provider": "[Provider Name]",
      "description": "[Brief description of the resource]"
    }},
    {{
      "title": "[Resource Title]",
      "provider": "[Provider Name]",
      "description": "[Brief description of the resource]"
    }},
    {{
        ...
    }}
  ]
}}]"""

few_shot_prompt = examples + "\n" + main_query

search_response = client.models.generate_content(
    model=model_id,
    contents=few_shot_prompt,
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"]
    )
)

# Parse and Format Learning Resources from Search Results

In [ ]:
# Get the raw text from the response
response_text = search_response.candidates[0].content.parts[0].text

# Extract JSON from markdown code blocks if present
if "```json" in response_text:
    json_text = response_text.split("```json")[1].split("```")[0].strip()
elif "```" in response_text:
    json_text = response_text.split("```")[1].split("```")[0].strip()
else:
    json_text = response_text

# Parse the JSON
try:
    resources_data = json.loads(json_text)
    
    # Format the resources as markdown
    resources_markdown = ""
    
    for item in resources_data:
        skill = item.get("skill", "")
        resources_markdown += f"**{skill}**\n"
        
        for resource in item.get("resources", []):
            title = resource.get("title", "")
            provider = resource.get("provider", "")
            description = resource.get("description", "")
            
            resources_markdown += f"* **{title} ({provider}):** {description}\n"
        
        resources_markdown += "\n"
    
except Exception as e:
    resources_markdown = f"Error processing resources: {str(e)}\n\nRaw response: {response_text}"


# Display Roadmap + Grounded Search Results

In [ ]:
from IPython.display import display, Markdown, HTML

# Create the complete markdown output
markdown_output = f"""
## Career Transition Plan

### Target Role: {target_role}
### Sector: {target_sector}

### 💱 Transferable Skills (with evidence):
{chr(10).join([f"- {skill['name']} ({skill['evidence']})" for skill in career_plan['transferable_skills']])}

### 🏋 Skills to Learn:
{chr(10).join([f"- {skill}" for skill in career_plan['missing_skills']])}

### 📜 Recommended Certifications:
{chr(10).join([f"- {cert}" for cert in career_plan['recommended_certifications']])}

### 👍 Recommended Learning Resources:
{chr(10).join([f"- {resource}" for resource in career_plan['learning_resources']])}

### 🚶‍➡️ Career Pathway:
{" -> ".join(career_plan['intermediate_roles'] + [target_role])}

### 🗺️ Roadmap Summary:
{career_plan['roadmap_summary']}

---

### 🔍 Skill Development Opportunities (via Google Search Grounding)
{resources_markdown}
"""

display(Markdown(markdown_output))

# Conclusion & Reflections

In this notebook, we explored how Generative AI can:
- Analyze structured and semi-structured content (PDF or LinkedIn profile)
- Compare a user's experience to a target role and identify transferable and missing skills
- Generate personalized career roadmap outputs from Gemini in structured JSON
- Use Google's built-in search grounding to recommend up-to-date learning resources using few-shot prompting to guide results

This application can support students and job seekers in navigating career pivots with clarity and confidence. Future expansions could include resume editing, interview prep, or integration with job boards and online learning platforms.